# Finetuning Whisper-large-V2 on Colab using PEFT-Lora training

In this Colab, we present a step-by-step guide on how to fine-tune Whisper for any multilingual ASR dataset using Hugging Face 🤗 Transformers and 🤗 PEFT. 

For more details on model, datasets and metrics, refer blog [Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-whisper)



## Inital Setup

In [5]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

/bin/bash: add-apt-repository: command not found
/bin/bash: apt: command not found
/bin/bash: apt: command not found


In [1]:
import os
os.environ['HF_HOME'] = '/scratch/s5521904/thesis/cache/'

In [2]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install -q bitsandbytes datasets accelerate
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git@main

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-gcbi_i8r
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-gcbi_i8r
  Resolved https://github.com/huggingface/transformers to commit 0d84901cb7e797c90653e2c8ca2ce2a6b3498208
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9019458 sha256=f214e376d2f85438602ce358fd94228d51e35b50e92c46ee537c281c1f06a5a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbap4qbd/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully

Linking the notebook to the Hub is straightforward - it simply requires entering your Hub authentication token when prompted. Find your Hub authentication token [here](https://huggingface.co/settings/tokens):

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# Select CUDA device index
import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['HF_HOME'] = '/scratch/s5521904/thesis/cache/'
model_name_or_path = "openai/whisper-small"
language = "English"
language_abbr = "en"
task = "transcribe"
#dataset_name = "mozilla-foundation/common_voice_6_1"


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Mon Jun  3 22:47:07 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.14              Driver Version: 550.54.14      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:17:00.0 Off |                    0 |
| N/A   33C    P0             35W /  250W |    7793MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

## Load Dataset

In [4]:
from datasets import load_dataset, load_metric, Audio
dataset = load_dataset("arrow", data_files={"train": '/scratch/s5521904/thesis/librispeech_1h/train/data-00000-of-00001.arrow',
                                           "test": ['/scratch/s5521904/thesis/extracted_librispeech/test/data-00000-of-00002.arrow',
                                                    '/scratch/s5521904/thesis/extracted_librispeech/test/data-00001-of-00002.arrow']})
                                     

In [65]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 309
    })
    test: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 2444
    })
})


## Prepare Feature Extractor, Tokenizer and Data

In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [6]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language= 'english', task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language='english', task=task)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Prepare Data

In [10]:
print(dataset["train"][0])
print(dataset["test"][0])

{'audio': {'path': None, 'array': array([-6.43920898e-03, -8.08715820e-03, -2.92968750e-03, ...,
       -1.58691406e-03,  6.10351562e-05,  1.34277344e-03]), 'sampling_rate': 16000}, 'text': 'THE DECKED OUT DANDIES BEFORE THE WORLD ARE FREQUENTLY THE GREATEST SLOVENS IN DOMESTIC PRIVACY HE WISHED HER GOOD MORNING IN A CARELESS TONE OF APATHY AND SHE AS APATHETICALLY ANSWERED TO IT PIERRE SAYS THERE ARE SOME LETTERS HE BEGAN', 'duration': 14.95}
{'audio': {'path': None, 'array': array([ 7.93457031e-04,  5.49316406e-04,  6.71386719e-04, ...,
        0.00000000e+00,  3.05175781e-05, -3.05175781e-05]), 'sampling_rate': 16000}, 'text': 'THY WAYS GREATLY TRY ME RUTH AND ALL THY RELATIONS', 'duration': 4.65}


Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 309
    })
    test: Dataset({
        features: ['audio', 'text', 'duration'],
        num_rows: 2444
    })
})


Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [13]:
print(dataset["train"][1])

{'audio': {'path': None, 'array': array([0.01519775, 0.02438354, 0.02801514, ..., 0.00616455, 0.01681519,
       0.00411987]), 'sampling_rate': 16000}, 'text': "LAST OUT A WHOLE TEN MONTHS YOU ARE IN EGREGIOUS ERROR COMPLIMENTS THE VERY OPPOSITE TO HONEY AND SWEETNESS HAVE GENERALLY SUPERVENED LONG BEFORE TRY IT IF YOU DON'T BELIEVE ME TWO LETTERS SHE CONTINUED", 'duration': 14.315}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [8]:
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    
    normalized_text = normalizer(batch["text"])

    # encode target text to label ids
    batch["labels"] = tokenizer(normalized_text).input_ids
    
    
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [9]:
dataset = dataset.map(prepare_dataset,remove_columns=dataset.column_names["train"], num_proc=2)

In [11]:
dataset["train"]

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 309
})

## Training and Evaluation

### Define a Data Collator

In [10]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [12]:
import evaluate

metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [13]:
from whisper.normalizers import EnglishTextNormalizer

normalizer = EnglishTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    pred_str_clean = [normalizer(pred_str) for pred in pred_str]
    label_str_clean = [normalizer(label_str) for label in label_str]

    wer = 100 * metric.compute(predictions=pred_str_clean, references=label_str_clean)

    return {"wer": wer}


### Load a Pre-Trained Checkpoint

Now let's load the pre-trained Whisper `small` checkpoint. Again, this
is trivial through use of 🤗 Transformers!

In [21]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, device_map="auto")

# model.hf_device_map - this should be {" ": 0}

Override generation arguments - no tokens are forced as decoder outputs (see [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), no tokens are suppressed during generation (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [22]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Post-processing on the model

Finally, we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast all non `int8` layers in `float32` for stability.

In [23]:
#comment out
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

### Apply LoRA

Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.

In [24]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 245,273,856 || trainable%: 1.442854145857274


In [23]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=16, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 1,880,064 || all params: 243,614,976 || trainable%: 0.771735806586866


In [24]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=2, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 331,776 || all params: 242,066,688 || trainable%: 0.13705975107157248


In [29]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=2, lora_alpha=64, target_modules=["q_proj", "v_proj", "o_proj", "k_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 331,776 || all params: 242,066,688 || trainable%: 0.13705975107157248


In [25]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=64, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 7,188,480 || all params: 248,923,392 || trainable%: 2.887828235925694


In [33]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=8, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 1,327,104 || all params: 243,062,016 || trainable%: 0.545993990274482


In [28]:
#from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

#config = LoraConfig(r=128, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

#model = get_peft_model(model, config)
#model.print_trainable_parameters()

trainable params: 14,155,776 || all params: 255,890,688 || trainable%: 5.531962147837127


We are ONLY using **1%** of the total trainable parameters, thereby performing **Parameter-Efficient Fine-Tuning**

### Define the Training Configuration

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [25]:

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="English_32r_LoRA_1h_new",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=50,
    num_train_epochs=30,
    evaluation_strategy="epoch",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=25,
    report_to=["tensorboard"],
    #load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)

**Few Important Notes:**
1. `remove_unused_columns=False` and `label_names=["labels"]` are required as the PeftModel's forward doesn't have the signature of the base model's forward.

2. INT8 training required autocasting. `predict_with_generate` can't be passed to Trainer because it internally calls transformer's `generate` without autocasting leading to errors.

3. Because of point 2, `compute_metrics` shouldn't be passed to `Seq2SeqTrainer` as seen below. (commented out)

In [26]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
#1e-3, 30epoch, 
#1e-4
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.714400,1.489977
2,0.479300,0.592536
3,0.347800,0.514796
4,0.255200,0.431392
5,0.196400,0.152388
6,0.028100,0.153404
7,0.025400,0.153701
8,0.013300,0.161264
9,0.011000,0.160094
10,0.008800,0.161901


TrainOutput(global_step=1170, training_loss=0.0980677171331695, metrics={'train_runtime': 6965.4179, 'train_samples_per_second': 1.331, 'train_steps_per_second': 0.168, 'total_flos': 2.7224273166336e+18, 'train_loss': 0.0980677171331695, 'epoch': 30.0})

In [28]:
kwargs = {
    "dataset_tags": "librispeech",
    "dataset": "librispeech",  # a 'pretty' name for the training dataset
    "dataset_args": "config: english, split: clean",
    #"language": "frisian",
    "model_name": "Whisper Small English 1h",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

In [29]:
trainer.push_to_hub(**kwargs)

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

events.out.tfevents.1716982555.a100gpu3.3419786.1:   0%|          | 0.00/23.7k [00:00<?, ?B/s]

events.out.tfevents.1716981721.a100gpu3.3419786.0:   0%|          | 0.00/7.58k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/xuliu15/English_32r_LoRA_1h_new/commit/3b4c8bf902f8a085b2c60b4584e78238180c100b', commit_message='End of training', commit_description='', oid='3b4c8bf902f8a085b2c60b4584e78238180c100b', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
model_name_or_path = "openai/whisper-small"
peft_model_id = "xuliu15/" + f"{model_name_or_path}-English-32r-1h-new".replace("/", "-")
#peft_model_id = "cantonese/" + f"{model_name_or_path}-{model.peft_config.PeftType}-colab".replace("/", "-")

model.push_to_hub(peft_model_id)
print(peft_model_id)



README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

xuliu15/openai-whisper-small-English-32r-1h-new


# Evaluation and Inference

**Important points to note while inferencing**:
1. As `predict_with_generate` can't be used, we will write the eval loop with `torch.cuda.amp.autocast()` as shown below.
2. As the base model is frozen, PEFT model sometimes fails ot recognise the language while decoding.Hence, we force the starting tokens to mention the language we are transcribing. This is done via `forced_decoder_ids = processor.get_decoder_prompt_ids(language="Marathi", task="transcribe")` and passing that to the `model.generate` call.
3. Please note that [AutoEvaluate Leaderboard](https://huggingface.co/spaces/autoevaluate/leaderboards?dataset=mozilla-foundation%2Fcommon_voice_11_0&only_verified=0&task=automatic-speech-recognition&config=mr&split=test&metric=wer) for `mr` language on `common_voice_11_0` has a bug wherein openai's `BasicTextNormalizer` normalizer is used while evaluation leading to degerated output text, an example is shown below:
```
without normalizer: 'स्विच्चान नरुवित्तीची पद्दत मोठ्या प्रमाणात आमलात आणल्या बसोन या दुपन्याने अनेक राथ प्रवेश केला आहे.'
with normalizer: 'स व च च न नर व त त च पद दत म ठ य प रम ण त आमल त आणल य बस न य द पन य न अन क र थ प रव श क ल आह'
```
Post fixing this bug, we report the 2 metrics for the top model of the leaderboard and the PEFT model:
1. `wer`: `wer` without using the `BasicTextNormalizer` as it doesn't cater to most indic languages. This is want we consider as true performance metric.
2. `normalized_wer`: `wer` using the `BasicTextNormalizer` to be comparable to the leaderboard metrics.
Below are the results:

| Model          | DrishtiSharma/whisper-large-v2-marathi | smangrul/openai-whisper-large-v2-LORA-colab |
|----------------|----------------------------------------|---------------------------------------------|
| wer            | 35.6457                                | 36.1356                                     |
| normalized_wer | 13.6440                                | 14.0165                                     |

We see that PEFT model's performance is comparable to the fully fine-tuned model on the top of the leaderboard. At the same time, we are able to train the large model in Colab notebook with limited GPU memory and the added advantage of resulting checkpoint being jsut `63` MB.



In [31]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "xuliu15/English_32r_LoRA_1h_new"

peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

In [32]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc
#from transformers import Seq2SeqGenerationConfig
eval_dataloader = DataLoader(dataset["test"], batch_size=8, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    decoder_input_ids=batch["labels"][:, :4].to("cuda"),
                    max_new_tokens=255,
                    language = "English"
                    
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
#             decoded_preds = normalizer(tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            
            
            
            normalized_preds = [normalizer(pred) for pred in decoded_preds]
            #normalized_labels = [normalizer(label) for label in decoded_labels]
            print("prediction:", normalized_preds)
            print("truth:", decoded_labels)
            metric.add_batch(
                
                predictions=normalized_preds,
                references=decoded_labels,
            )
    del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute()
print(f"{wer=}")

  0%|          | 0/306 [00:00<?, ?it/s]

prediction: ['thy ways greatly try me ruth and all thy relations', 'is he going to start a daily newspaper among the kickapoo is', 'has the consulted thy mother about a career i suppose it is a career they want', 'but neither saint jurard nor broad street neither wonders of the ment nor the glories of the hall were the ghosts of our fathers sit always signing the declaration impressed the visitors so much as the splendors of the chestnut street windows and the bargains on 8th street', 'is thy father willing these should go away to a school of the world is people', 'does d think d could stand it 6 months', 'i have not asked him ruth replied with a look that might imply that she was one of those determined little bodies who 1st made up her own mind and then compelled others to make up theirs in accordance with hers', 'i have nothing to wear replied that demure person']
truth: ['thy ways greatly try me ruth and all thy relations', 'is he going to start a daily newspaper among the kick a p

  1%|          | 2/306 [00:07<18:20,  3.62s/it]

prediction: ['the study medicine', 'if i go to meeting at all i like best to sit in the quiet old house in german town where the windows are all open and i can see the trees and hear the stir of the leaves', 'that filip is honest and he has talent enough if he will stop scribbling to make his way', 'i heard father tell cousin abner that he was whipped so often for whistling when he was a boy that he was determined to have what compensation he could get now', 'mother i am going to study medicine', 'and then there was broad street', 'ruth sat quite still for a time with face and tent and flushed it was out now', 'well mother said the young student looking up with the shade of impatience']
truth: ['thee study medicine', 'if i go to meeting at all i like best to sit in the quiet old house in germantown where the windows are all open and i can see the trees and hear the stir of the leaves', 'but philip is honest and he has talent enough if he will stop scribbling to make his way', 'i heard 

  1%|          | 3/306 [00:09<13:47,  2.73s/it]

...

100%|█████████▉| 305/306 [08:38<00:01,  1.42s/it]

prediction: ['saturday august 15th the sea unbroken all round no land in sight', '2 hours afterwards a terrible shock awoke me', 'nothing new weather unchanged the wind freshens', 'i take this is my answer and i leave the professor to bite his lips with impatience', 'as for the itheusaurus has he returned to his submarine cabin', 'truly the sea is of infinite width', 'but there seemed no reason of fear', 'therefore do not talk to me about views and prospects']
truth: ['saturday august 15th the sea unbroken all round no land in sight', '2 hours afterwards a terrible shock awoke me', 'nothing new weather unchanged the wind freshens', 'i take this as my answer and i leave the professor to bite his lips with impatience', 'as for the ichthyosaurus has he returned to his submarine cavern', 'truly this sea is of infinite width', 'but there seemed no reason to fear', 'therefore do not talk to me about views and prospects']


100%|██████████| 306/306 [08:39<00:00,  1.70s/it]

prediction: ['those huge creatures attacked each other with the greatest animosity', 'you seem anxious my uncle i said seeing him continually with his glass to his eye anxious', 'sunday august 16th', 'slow to world greetings quick with its 0 list when the angel speak']
truth: ['those huge creatures attacked each other with the greatest animosity', 'you seem anxious my uncle i said seeing him continually with his glass to his eye anxious', 'sunday august 16th', 'slow to world greetings quick with its 0 list when the angels speak']


wer=4.964524547382156
